In [17]:
import pandas as pd
import numpy as py
import pickle 

In [18]:
dataset = pd.read_csv("heart.csv")
dataset

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,45,M,TA,110,264,0,Normal,132,N,1.2,Flat,1
914,68,M,ASY,144,193,1,Normal,141,N,3.4,Flat,1
915,57,M,ASY,130,131,0,Normal,115,Y,1.2,Flat,1
916,57,F,ATA,130,236,0,LVH,174,N,0.0,Flat,1


In [19]:
df1 = dataset.copy()
#df1

In [20]:
df1.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

## 1. Checking the Null values...


In [21]:
df1.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
HeartDisease      0
dtype: int64

### No NULL values!

## 2. Outlier treatment
### Check outliers with IQR method...

In [22]:
## Seperation of Quan and Qual column...

quan =[]
qual = []
for column_name in dataset.columns:
    if dataset[column_name].dtype=="object":
        qual.append(column_name)
    else:
        quan.append(column_name)
    

In [23]:
quan

['Age',
 'RestingBP',
 'Cholesterol',
 'FastingBS',
 'MaxHR',
 'Oldpeak',
 'HeartDisease']

In [24]:
qual

['Sex', 'ChestPainType', 'RestingECG', 'ExerciseAngina', 'ST_Slope']

In [25]:
lower = []
upper = []

for col_name in quan:
    Q1 = dataset[col_name].quantile(0.25)
    Q3 = dataset[col_name].quantile(0.75)
    IQR = Q3 - Q1
    
    # Identify outliers below Q1 - 1.5 * IQR
    if (dataset[col_name] < (Q1 - 1.5 * IQR)).any():
        lower.append(col_name)
    
    # Identify outliers above Q3 + 1.5 * IQR
    if (dataset[col_name] > (Q3 + 1.5 * IQR)).any():
        upper.append(col_name)

# Compile outlier results in a dictionary
outliers_s = {
    "Lower Outliers": lower,
    "Upper Outliers": upper
}

print("Outliers in the features:\n\n", outliers_s)

Outliers in the features:

 {'Lower Outliers': ['RestingBP', 'Cholesterol', 'MaxHR', 'Oldpeak'], 'Upper Outliers': ['RestingBP', 'Cholesterol', 'FastingBS', 'Oldpeak']}


## Qutliers treatement- capping then with lower range and upper range...

In [26]:
# Qutliers treatement 

# Define the lower and upper quantiles
for col_name in quan:
    lower_range = dataset[col_name].quantile(0.25)
    upper_range = dataset[col_name].quantile(0.75)
    
    # Treat lower outliers by capping them at the lower 
    ### Select Rows with .loc
    
    ##### dataset.loc[<condition>, col_name]: This selects rows in the dataset where the condition dataset[col_name] < lower_range is True
    ##### = lower_range: This part of the statement sets all values in col_name that are below lower_range to exactly lower_range
    dataset.loc[dataset[col_name] < lower_range, col_name] = lower_range
    
    # Treat upper outliers by capping them at the upper range
    ##### dataset.loc[<condition>, col_name]: This selects rows in the dataset where the condition dataset[col_name] > upper_range is True
    dataset.loc[dataset[col_name] > upper_range, col_name] = upper_range

print("Dataset with outliers capped at range limits:\n", dataset)
    

Dataset with outliers capped at range limits:
      Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  \
0     47   M           ATA        140       267.00          0     Normal   
1     49   F           NAP        140       180.00          0     Normal   
2     47   M           ATA        130       267.00          0         ST   
3     48   F           ASY        138       214.00          0     Normal   
4     54   M           NAP        140       195.00          0     Normal   
..   ...  ..           ...        ...          ...        ...        ...   
913   47   M            TA        120       264.00          0     Normal   
914   60   M           ASY        140       193.00          0     Normal   
915   57   M           ASY        130       173.25          0     Normal   
916   57   F           ATA        130       236.00          0        LVH   
917   47   M           NAP        138       175.00          0     Normal   

     MaxHR ExerciseAngina  Oldpeak ST_Sl

/var/folders/1s/70xy4dvs6mn0zqfz6nvhbypw0000gn/T/ipykernel_36335/1161479449.py:13: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '173.25' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  dataset.loc[dataset[col_name] < lower_range, col_name] = lower_range


In [27]:
# Rechecking if any outliers...

lower = []
upper = []

for col_name in quan:
    Q1 = dataset[col_name].quantile(0.25)
    Q3 = dataset[col_name].quantile(0.75)
    IQR = Q3 - Q1
    
    # Identify outliers below Q1 - 1.5 * IQR
    if (dataset[col_name] < (Q1 - 1.5 * IQR)).any():
        lower.append(col_name)
    
    # Identify outliers above Q3 + 1.5 * IQR
    if (dataset[col_name] > (Q3 + 1.5 * IQR)).any():
        upper.append(col_name)

# Compile outlier results in a dictionary
outliers_s = {
    "Lower Outliers": lower,
    "Upper Outliers": upper
}

print("Outliers in the features:\n\n", outliers_s)

Outliers in the features:

 {'Lower Outliers': [], 'Upper Outliers': []}


In [28]:
dataset

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,47,M,ATA,140,267.00,0,Normal,156,N,0.0,Up,0
1,49,F,NAP,140,180.00,0,Normal,156,N,1.0,Flat,1
2,47,M,ATA,130,267.00,0,ST,120,N,0.0,Up,0
3,48,F,ASY,138,214.00,0,Normal,120,Y,1.5,Flat,1
4,54,M,NAP,140,195.00,0,Normal,122,N,0.0,Up,0
...,...,...,...,...,...,...,...,...,...,...,...,...
913,47,M,TA,120,264.00,0,Normal,132,N,1.2,Flat,1
914,60,M,ASY,140,193.00,0,Normal,141,N,1.5,Flat,1
915,57,M,ASY,130,173.25,0,Normal,120,Y,1.2,Flat,1
916,57,F,ATA,130,236.00,0,LVH,156,N,0.0,Flat,1


## 3.Categorical feature encoding using Onehot encoding-¶...

In [29]:
dataset_encoded_dtypes=pd.get_dummies(dataset,drop_first=True,dtype=int)
dataset_encoded_dtypes

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,47,140,267.00,0,156,0.0,0,1,1,0,0,1,0,0,0,1
1,49,140,180.00,0,156,1.0,1,0,0,1,0,1,0,0,1,0
2,47,130,267.00,0,120,0.0,0,1,1,0,0,0,1,0,0,1
3,48,138,214.00,0,120,1.5,1,0,0,0,0,1,0,1,1,0
4,54,140,195.00,0,122,0.0,0,1,0,1,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
913,47,120,264.00,0,132,1.2,1,1,0,0,1,1,0,0,1,0
914,60,140,193.00,0,141,1.5,1,1,0,0,0,1,0,0,1,0
915,57,130,173.25,0,120,1.2,1,1,0,0,0,1,0,1,1,0
916,57,130,236.00,0,156,0.0,1,0,1,0,0,0,0,0,1,0


In [30]:
# Save the file 
dataset_encoded_dtypes.to_csv('prep_ds.csv',index=False)